In [ ]:
# Satellite Image Analysis GGS416 - Week4

In this tutorial we are going to visualize a satellite image using the `pyplot` function from `matplotlib`.

It is important you should be able to visualize an image:
- In a piece of GIS software via a Graphical User Interface (GUI), such as in ESRI ArcMap or preferably an open-source alternative such as QGIS.
- Using a script-based approach via visualization packages such as `matplotlib`

To begin, we need to import one of our satellite images via `rasterio`, as follows:

# Compressing a file

Often we may need to compress a file to reduce the size of the given data. 

We have two options:
- **lossy** compression which is a method of data compression in which the size of the file is reduced by eliminating data in the file (thus, lowering quality).
- **lossless** which is a class of data compression algorithms that allows the original data to be perfectly reconstructed from the compressed data (thus, no loss of quality).

For satellite image process, lossy is typically not very useful as we suffer a reduction in the present quality (and quality is really important).

Many satellite data sources provide imagery as GeoTIFFs using lossless LZW compression (e.g. Planet). 

Let's see an example of a lossy method where we will copy an image and save it as a .jpeg. 

In [ ]:
import os

# Let's us our OS package to get the size of a given image file
size = os.path.getsize("20160831_180302_0e26_3B_Visual.tif")

print('The size of the file is {} bytes'.format(size))
print('This translates to {} Megabytes'.format(round(size/1e6, 1)))

In [ ]:
Next we need to: 
- Read our data. 
- Extract the metadata.
- Use the `Rasterio` write command. 

And then we can evaluate the size changes by converting to .jpeg:

In [ ]:
# read all bands from source dataset into a single 3-dimensional ndarray
data = my_image.read()

# write new file using profile metadata from original dataset
# and specifying JPEG compression

profile = my_image.profile
profile['compress'] = 'JPEG'

# You can tell this writes the data as we provide the 'w' command, which stands for 'write'
with rasterio.open('compressed.tif', 'w', **profile) as my_raster_writer:
    my_raster_writer.write(data)

new_size = os.path.getsize("compressed.tif")
print('The size of the file is {} bytes'.format(new_size))
print('This translates to {} Kilobytes'.format(round(new_size/1e3, 1)))

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} matplotlib

In [ ]:
# import rasterio
from matplotlib import pyplot as plt

# This notebook explores a single 4 band (blue, green, red, NIR) PlanetScope scene in a UTM projection.
image_file = "20190321_174348_0f1a_3B_AnalyticMS.tif"

# Use Rasterio to open the image.
satdat = rasterio.open(image_file)
satdat

# rasterio.plot.show is a shortcut to matplotlib plotting

from rasterio.plot import show
show(satdat)

"Using a mask to cut raster along boundary
In this exercise we're going to clip a fullsized PlanetScope dataset to an area of interest (AOI) polygon. Because this AOI is a vector dataset (versus raster), we'll use Rasterio's sister-library, Fiona, to read the geoJSON dataset:"

In [ ]:
!conda install --yes --prefix {sys.prefix} fiona

In [ ]:
import fiona 

with fiona.open("bbox.shp", "r") as shapefile:
    my_geojson = [feature["geometry"] for feature in shapefile]
my_geojson

In [ ]:
my_geojson = [{
	"type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
	"coordinates": [ #Here are our actual geometry coordinates
	  [
		[
		  559611.0942910484736785,
		  4183386.0969752739183605
		],
		[
		  562552.8939972541993484,
		  4183386.0969752739183605
		],
		[
		  562552.8939972541993484,
		  4186004.0364823746494949
		],
		[
		  559611.0942910484736785,
		  4186004.0364823746494949
		],
		[
		  559611.0942910484736785,
		  4183386.0969752739183605
		]
	  ],
	]
  }]
my_geojson

"In order to clip our original raster to the boundary defined by the AOI, we'll use rasterio's mask function. This will create a copy of our original dataset, with all pixels outside of the input AOI shape set to nodata values."

In [ ]:
# import rasterio's mask tool
from rasterio.mask import mask

# apply mask with crop=True to crop the resulting raster to the AOI's bounding box
with rasterio.open(image_file) as img:
    clipped, transform = mask(img, my_geojson, crop=True)
    
print(clipped.shape)

"Using a copy of the metadata from our original raster dataset, we can write a new geoTIFF containing the new, clipped raster data:"

In [ ]:
# use the metadata from our original mosaic
meta = img.meta.copy()

# update metadata with new, clipped mosaic's boundaries
meta.update({"transform": transform,
    "height":clipped.shape[1],
    "width":clipped.shape[2]})

# write the clipped-and-cropped dataset to a new GeoTIFF
with rasterio.open('clipped.tif', 'w', **meta) as dst:
    dst.write(clipped)

clipped_img = rasterio.open("clipped.tif")
show(clipped_img)

# Compressing a file

Often we may need to compress a file to reduce the size of the given data. 

We have two options:
- **lossy** compression which is a method of data compression in which the size of the file is reduced by eliminating data in the file (thus, lowering quality).
- **lossless** which is a class of data compression algorithms that allows the original data to be perfectly reconstructed from the compressed data (thus, no loss of quality).

For satellite image process, lossy is typically not very useful as we suffer a reduction in the present quality (and quality is really important).

Many satellite data sources provide imagery as GeoTIFFs using lossless LZW compression (e.g. Planet). 

Let's see an example of a lossy method where we will copy an image and save it as a .jpeg. 